<a href="https://colab.research.google.com/github/wenjing89/news/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 第一個步驟請利用聯合新聞網作為主要的文本來源，將範例收集回來的新聞內文存在一個List容器變數`news`內
import requests
from bs4 import BeautifulSoup

# 利用 requests 對 API 來源發送一個請求
response = requests.get('https://udn.com/news/breaknews/1')
soup = BeautifulSoup(response.text) # 利用 BeautifulSoup(...) 將取回來的資料清理成純文字後存到 `news_content`
news = []

for link in soup.find_all('h3', class_='rounded-thumb__title')[:4]:
  news_url = link.a['href']
  news_response = requests.get('https://udn.com' + news_url)
  news_soup = BeautifulSoup(news_response.text)
  news_content = news_soup.find('div', class_='article-content__paragraph').text.strip().replace('\n', ' ')
  news.append(news_content)
print(news)


['現年42歲、擁有「黑色閃電」綽號、台灣半馬紀錄保持人蔣介文，今被《鏡週刊》報導，與一名年約40歲的已婚張姓女子過從甚密，兩人從練跑到上旅館開房，男方甚至在公開場合中親吻女子臉頰，該名女子也笑臉回迎。由於全程被週刊直擊，求證雙方後，兩人都否認有不正常男女關係。 \r 拿過上百面田徑獎牌的蔣介文擅長中長跑，也在坊間開班授課，教學跑步技巧。但週刊卻直擊，蔣介文幾乎每週二固定會從台中北上陪跑一名已婚女子，且兩人還在正式練跑前數小時，先赴商辦大樓內的旅館開房約會，在公共場合中則不避諱做出親暱動作，包括蔣會用手溫柔地擦掉女子臉上的異物、主動撐傘，甚至在夜間的居酒屋吧檯上，主動親吻張女臉頰，張女則未迴避，全程笑臉以對。            \r 據報導稱，該名張姓女子現職為台北某知名證券公司的上班族。兩人在週刊查證的過程中，均否認有不倫戀之實。蔣介文並強調他在去（2023）年年底已離婚，現為單身；張女則表示，與蔣並非是情侶關係，兩人約會曝光後，也嚴重影響到她的婚姻，目前已到警局備案。 \r 報導還披露，蔣介文的前妻在去年因為察覺先生疑似不忠，還在IG上擷取張姓女子感謝蔣教練的照片，並附上「蔣你媽啦！」表達不滿，且在蔣介文生日時貼出他們一雙兒女的合影，並隨圖註明「我有證據」，顯示兩人在去年時已關係緊張。 台灣半馬紀錄保持人蔣介文被週刊爆出不倫。擷自蔣介文臉書       外遇 馬拉松', '12生肖每個生肖都有自己獨特的性格和命運走向，「搜狐」列出有3個生肖的子女往往被認為是最不關心父母的，被指容易成為「白眼狼（比喻忘恩負義）」，分別是鼠、雞、猴，為什麼這3生肖的子女會對父母如此冷漠呢？我們來一一探討。  鼠 \r 生肖鼠的人天生聰明伶俐、獨立自主，從小就懂得如何為自己爭取利益，追求個人價值。在成長的過程中，他們往往會把個人的發展和成功看得比家庭更重要，這並不是說他們都不關心父母，而是他們更注重自己的自我實現和獨立生活。             \r 他們渴望擁有更多的自由和空間，以便追求自己的夢想和目標，因此，有時候他們可能會表現出一種冷漠的態度，讓父母感到被忽視。  雞\r 生肖雞的人天生具有強烈的競爭意識和上進心，他們渴望在事業上取得卓越成就，這種強烈的追求使得他們在面對家庭責任時，可能會顯得疏遠，他們往往會把時間投入在工作中，透過事業的成功來證明自己的價值。 \r 這

In [11]:
# 安裝 jieba 中文斷詞套件
!pip install jieba

# 下載官方字典檔
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big


# 載入套件與字典檔
import jieba
jieba.load_userdict("dict.txt.big")

--2024-04-24 13:13:41--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘dict.txt.big.2’

dict.txt.big.2      100%[===================>]   8.18M  --.-KB/s    in 0.06s   

2024-04-24 13:13:42 (145 MB/s) - ‘dict.txt.big.2’ saved [8583143/8583143]



In [15]:
# 將每篇新聞文章斷詞後的結果存入 `tokens` 變數，並且將斷詞後的結果計算成每個單字的出現次數（詞頻）
# 並存成以「出現單字為 KEY、出現次數為 Value」的 dict 型態變數 `word_count`。

# 精確模式斷詞
tokens = []

for d in news:
  token = list(jieba.cut(d, HMM=False)) # token = d 字串斷詞結果
  tokens.append(token)
#print(tokens)

word_count = {}
for word in tokens:
  for num in word:
    if num not in word_count:
      word_count[num] = 1
    else:
      word_count[num] += 1

print(word_count)
# {'聲明': 4, '主管機關': 3, ...}


{'現年': 1, '42': 1, '歲': 3, '、': 25, '擁有': 3, '「': 27, '黑色': 1, '閃電': 1, '」': 27, '綽號': 1, '台': 10, '灣': 4, '半': 2, '馬': 5, '紀錄': 2, '保持': 3, '人': 16, '蔣': 12, '介文': 8, '，': 169, '今': 1, '被': 8, '《': 1, '鏡': 1, '週刊': 5, '》': 1, '報導': 3, '與': 5, '一名': 4, '年': 5, '約': 1, '40': 1, '的': 80, '已婚': 2, '張': 6, '姓': 3, '女子': 7, '過從甚密': 1, '兩': 6, '從': 2, '練跑': 2, '到': 5, '上': 6, '旅館': 2, '開房': 2, '男方': 1, '甚至': 2, '在': 27, '公開場合': 1, '中': 7, '親吻': 2, '臉頰': 2, '該': 3, '名': 2, '也': 12, '笑臉': 2, '回': 1, '迎': 1, '。': 34, '由於': 3, '全程': 2, '直': 2, '擊': 2, '求證': 1, '雙方': 1, '後': 6, '都': 6, '否認': 2, '有': 9, '不': 12, '正常': 1, '男女關係': 1, ' ': 129, '\r': 19, '拿': 1, '過': 3, '上百': 1, '面': 1, '田徑': 1, '獎牌': 1, '擅長': 1, '中長跑': 1, '坊間': 1, '開班': 1, '授課': 1, '教學': 1, '跑步': 1, '技巧': 1, '但': 7, '卻': 2, '幾乎': 1, '每週': 1, '二': 4, '固定': 1, '會': 18, '北上': 1, '陪': 1, '跑': 1, '且': 2, '還': 7, '正式': 1, '前': 3, '數': 1, '小時': 1, '先': 1, '赴': 3, '商辦': 1, '大樓': 1, '內': 1, '約會': 2, '公共場合': 1, '則': 6, '避諱': 1, '做出': 1, '親': 

In [18]:
# 將每個單字的出現次數（詞頻）依照出現次數由多到少進行排序
word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))

print(word_count)

{'，': 169, ' ': 129, '的': 80, '。': 34, '「': 27, '」': 27, '在': 27, '、': 25, '\r': 19, '會': 18, '他們': 18, '自己': 17, '人': 16, '是': 16, '和': 16, '為': 15, '父母': 15, '兩岸': 15, '蔣': 12, '也': 12, '不': 12, '了': 12, '大陸': 11, '台': 10, '生肖': 10, '有': 9, '介文': 8, '被': 8, '錢': 8, '女子': 7, '中': 7, '但': 7, '還': 7, '習近平': 7, '張': 6, '兩': 6, '上': 6, '後': 6, '都': 6, '則': 6, '去': 6, '子女': 6, '什麼': 6, '？': 6, '家庭': 6, '多': 6, '可能': 6, '爸媽': 6, '同胞': 6, '走': 6, '旅遊': 6, '馬': 5, '週刊': 5, '與': 5, '年': 5, '到': 5, '對': 5, '並': 5, '他': 5, '關心': 5, '我們': 5, '就': 5, '追求': 5, '發展': 5, '自由': 5, '很': 5, '原': 5, 'PO': 5, '算': 5, '國': 5, '灣': 4, '一名': 4, '二': 4, '已': 4, '；': 4, '你': 4, '時': 4, '往往': 4, '最': 4, '雞': 4, '猴': 4, '來': 4, '個人': 4, '因此': 4, '責任': 4, '網友': 4, '文': 4, '如今': 4, '想要': 4, '又': 4, '小孩': 4, '而': 4, '需要': 4, '跟': 4, '身': 4, '要': 4, '一': 4, '沒有': 4, '日': 4, '辦': 4, '合作': 4, '談話': 4, '越': 4, '新': 4, '歲': 3, '擁有': 3, '保持': 3, '報導': 3, '姓': 3, '該': 3, '由於': 3, '過': 3, '前': 3, '赴': 3, '包括': 3, '女': 3, '